In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46943
2,Huelva,Confirmados PDIA 14 días,620
3,Huelva,Tasa PDIA 14 días,"120,81766276282713"
4,Huelva,Confirmados PDIA 7 días,318
5,Huelva,Tasa PDIA 7 dias,"61,96776896545005"
6,Huelva,Total Confirmados,47146
7,Huelva,Curados,45573
8,Huelva,Fallecidos,422


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46943.0


/tmp/ipykernel_48036/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12965.0


/tmp/ipykernel_48036/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_48036/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_48036/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_48036/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1035 personas en los últimos 7 días 

Un positivo PCR cada 604 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46943.0,318.0,620.0,513170.0,61.967769,120.817663,83.0
Huelva-Costa,27761.0,206.0,366.0,289548.0,71.145371,126.403912,55.0
Huelva (capital),12965.0,139.0,238.0,143837.0,96.637166,165.465075,36.0
Condado-Campiña,14620.0,102.0,217.0,156231.0,65.287939,138.896890,25.0
Cartaya,2211.0,37.0,64.0,20083.0,184.235423,318.677488,11.0
Palos de la Frontera,1144.0,28.0,60.0,11742.0,238.460228,510.986203,8.0
Moguer,2075.0,16.0,34.0,21867.0,73.169616,155.485435,5.0
Aljaraque,1663.0,12.0,16.0,21474.0,55.881531,74.508708,5.0
Palma del Condado (La),1458.0,14.0,22.0,10801.0,129.617628,203.684844,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granada de Río-Tinto (La),7.0,1.0,2.0,243.0,411.522634,823.045267,1.0
Palos de la Frontera,1144.0,28.0,60.0,11742.0,238.460228,510.986203,8.0
Cartaya,2211.0,37.0,64.0,20083.0,184.235423,318.677488,11.0
Bollullos Par del Condado,1334.0,14.0,36.0,14387.0,97.310072,250.225898,1.0
San Bartolomé de la Torre,397.0,6.0,9.0,3761.0,159.532039,239.298059,2.0
Palma del Condado (La),1458.0,14.0,22.0,10801.0,129.617628,203.684844,4.0
Paterna del Campo,218.0,3.0,7.0,3457.0,86.780445,202.487706,2.0
Huelva (capital),12965.0,139.0,238.0,143837.0,96.637166,165.465075,36.0
Moguer,2075.0,16.0,34.0,21867.0,73.169616,155.485435,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Sierra de Huelva-Andévalo Central,4134.0,9.0,35.0,67391.0,13.354899,51.935718,3.0,0.257143
Isla Cristina,3194.0,8.0,24.0,21393.0,37.395410,112.186229,0.0,0.333333
Gibraleón,962.0,4.0,12.0,12737.0,31.404569,94.213708,0.0,0.333333
Almonte,2362.0,9.0,24.0,24507.0,36.724201,97.931203,2.0,0.375000
Bollullos Par del Condado,1334.0,14.0,36.0,14387.0,97.310072,250.225898,1.0,0.388889
Nerva,259.0,2.0,5.0,5169.0,38.692204,96.730509,0.0,0.400000
Valverde del Camino,864.0,2.0,5.0,12750.0,15.686275,39.215686,1.0,0.400000
Paterna del Campo,218.0,3.0,7.0,3457.0,86.780445,202.487706,2.0,0.428571
Palos de la Frontera,1144.0,28.0,60.0,11742.0,238.460228,510.986203,8.0,0.466667
